In [24]:
from fastapi import FastAPI
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
from recommender_item_item import item_item_recom

app = FastAPI()

#http://127.0.0.1:8000

#-----------------------------------------INICIO-------------------------------------------#
@app.get("/")
def index():
    return 'API desarrollada para el PI1 MLOps por Alter Caimi'

df_steam = pd.read_parquet('../CleanData/steam_games.parquet')
df_user = pd.read_parquet('../CleanData/users_items.parquet')
df_reviews = pd.read_parquet('../CleanData/reviews.parquet')
df_sent = pd.read_parquet('../sentiment_analysis_2.parquet')

#-----------------------------------------ENDPOINT 1---------------------------------------#
@app.get('/developer/{desarrollador}')
def developer(desarrollador: str):
    
    """
    Recupera información sobre un desarrollador específico.

    Args:
        desarrollador (str): Nombre del desarrollador.

    Returns:
        pd.DataFrame: DataFrame con información sobre el desarrollador.
    """
    
    if not isinstance(desarrollador, str):
        return {'Mensaje': 'El argumento "desarrollador" debe ser una cadena de texto (str).'}
    
    df_steam['free'] = df_steam['price'].apply(lambda x: 1 if x == 0 else 0)
    df_steam['Año'] = df_steam['release_date'].dt.year
    
    desarrolladores = df_steam['developer'].str.lower().unique()
    desarrollador = desarrollador.lower()

    if desarrollador not in desarrolladores:
        return {'Mensaje': 'Desarrollador no encontrado. Inserte un desarrollador válido'}

    df = df_steam.groupby(['Año', 'developer']).agg(
                            {'id': 'count', 'free': lambda x: (x.sum() / x.count()) * 100}
                        ).reset_index().rename(columns={'free': 'Contenido Free', 'id': 'Cantidad de Items'})
    
    df['developer'] = df['developer'].apply(lambda x: x.lower())
    
    df = df[df['developer'] == desarrollador].drop(columns= 'developer').sort_values(by= 'Año', ascending=False)
    df['Contenido Free'] = round(df['Contenido Free'], 2)
    df['Contenido Free'] = df['Contenido Free'].apply(lambda x: str(x) + '%')
    df.reset_index(drop=True, inplace=True)

    resultado = {
        f'Año {int(df.loc[i,"Año"])}': {'Cantidad de Items': int(df.loc[i, 'Cantidad de Items']), 'Contenido Free:': df.loc[i, 'Contenido Free']}
        for i in range(len(df))
    }

    return resultado

#-----------------------------------------ENDPOINT 2---------------------------------------#
@app.get('/userdata/{user_id}')
def userdata(user_id: str):

    if not isinstance(user_id, str):
        return {'Mensaje': 'El argumento user_id debe ser una cadena de texto.'}

    df_user_n = df_user[['user_id', 'item_id', 'items_count']]
    
    usuarios = df_user_n['user_id'].unique()

    if user_id not in usuarios:
        return {'Mensaje': 'Usuario no encontrado. Por favor ingrese un usuario válido'}
    
    df_steam_n = df_steam[['id', 'price']]
    df_reviews_n = df_reviews[['user_id', 'recommend']]
    
    df_reviews_n = df_reviews_n.groupby('user_id').agg('sum').reset_index()

    df = df_user_n.merge(df_steam_n, how='left', left_on= 'item_id', right_on= 'id')
    df = df.merge(df_reviews_n, how = 'left')
    df = df.drop(columns=['item_id', 'id'])
    df = df.groupby('user_id').agg('max').reset_index()
    df = df[df['user_id'] == user_id]

    df['recommend'] = df['recommend'].fillna(0)
    df['recommend'] = round(df['recommend'] / df['items_count'], 2)
    df['recommend'] = df['recommend'].apply(lambda x: str(x) + ' %')
    df.rename(columns= {'recommend': '% de recomendación'}, inplace=True)
    
    df.reset_index(inplace=True, drop=True)

    resultado = {
        'Usuario': df.loc[0, 'user_id'],
        'Dinero gastado': str(df.loc[0, 'price']) + 'USD',
        '% de recomendación': df.loc[0, '% de recomendación'],
        'Cantidad de Items': int(df.loc[0, 'items_count'])
    }
    return resultado

#-----------------------------------------ENDPOINT 3---------------------------------------#
@app.get('/UserForGenre/{genero}')
def UserForGenre(genero: str):
    
    if not isinstance(genero, str):
        return {'Mensaje': 'El género ingresado debe ser una cadena de texto (string)'}

    genero = 'genre_' + genero
    genero = genero.lower()

    df_steam_n = df_steam.rename(columns= lambda x: x.lower())

    columnas = list(df_steam_n.columns)

    if genero not in columnas:
        generos = [col.replace('genre_', '') for col in columnas if 'genre_' in col]
        return {'Mensaje': 'Género no encontrado. Ingrese un género válido',
                'Géneros disponibles': generos}

    df_user_n = df_user[['user_id', 'item_id', 'playtime_forever']]

    df_steam_n = df_steam_n[df_steam_n[genero] == 1][['id','release_date']]
    df_steam_n['Año'] = df_steam_n['release_date'].dt.year
    df_steam_n.drop(columns= 'release_date', inplace= True)

    df = df_user_n.merge(df_steam_n, how= 'left', left_on= 'item_id', right_on= 'id')
    df = df.dropna()
    df = df.groupby(['user_id', 'Año']).agg({'playtime_forever': 'sum'}).reset_index()
    df['playtime_forever'] = round(df['playtime_forever']/60, 2)

    df_1 = df.groupby('user_id').agg({'playtime_forever' : 'sum'}).reset_index()
    ind = df_1['playtime_forever'].idxmax()
    usuario = df_1.loc[ind, 'user_id']
    df = df[df['user_id'] == usuario]
    df.rename(columns= {'playtime_forever': 'Horas jugadas'}, inplace=True)

    df.reset_index(inplace=True, drop=True)
    
    resultado = {
        f'Usuario con mas horas jugadas para el género {genero.replace("genre_", "")}:': df.loc[0,'user_id'],
        'Horas jugadas:': [{'Año:': int(df.loc[i,'Año']), 'Horas:': int(df.loc[i,'Horas jugadas'])} for i in range(len(df))]
    }

    return resultado

#-----------------------------------------ENDPOINT 4---------------------------------------#
@app.get('/best_developer_year/{anio}')
def best_developer_year(anio: int):

    try:
        anio = int(anio)
    except Exception as e:
        return {f'Error {e}': 'Debe insertar un número entero.'}
    
    df_sent_n = df_sent[['item_id', 'sentiment_analysis_2', 'recommend', 'Año']]

    anios = list(df_sent['Año'].unique())

    if anio not in anios:
        anios = [int(x) for x in anios]
        return {'Mensaje': f'No hay registros del año {anio}',
                'Los años disponibles son:': anios}
    
    df_steam_n = df_steam[['id', 'developer']]

    df = df_sent_n.merge(df_steam_n, how='left', left_on='item_id', right_on='id')
    df.drop(columns=['item_id', 'id'], inplace=True)
    df.rename(columns= {'sentiment_analysis_2': 'rating'}, inplace=True)

    df = df.groupby(['developer', 'Año']).agg({'rating': (lambda x: (x == 2).sum()), 'recommend': 'sum'}).reset_index()
    df = df[df['Año'] == anio]
    
    df['puntaje'] = df['rating'] + df['recommend']

    df.sort_values(by= 'puntaje', ascending=False, inplace=True)
    df.reset_index(inplace=True, drop= True)
    
    result = {
                'Puesto 1': df.loc[0, 'developer'],
                'Puesto 2': df.loc[1, 'developer'],
                'Puesto 3': df.loc[2, 'developer']
            }

    return result

#-----------------------------------------ENDPOINT 5---------------------------------------#
@app.get('/developer_reviews_analysis/{desarrolladora}')
def developer_reviews_analysis(desarrolladora: str):

    if not isinstance(desarrolladora, str):
        return {'Mensaje': 'Debe ingresar una cadena de texto'}
    
    desarrolladora = desarrolladora.lower()
    
    df_steam_n = df_steam[['id', 'developer']]
    df_steam_n['developer'] = df_steam_n['developer'].apply(lambda x: x.lower())

    developers = list(df_steam_n['developer'].unique())
    
    if desarrolladora not in developers:
        return {'Mensaje': 'Desarrolladora no encontrada. Por favor ingrese una desarrolladora válida.',
                'Desarrolladoras disponibles': developers}
    
    df_sent_n = df_sent[['item_id', 'sentiment_analysis_2']]

    df = df_sent_n.merge(df_steam_n,how= 'left', left_on= 'item_id', right_on='id')
    df.drop(columns=['id', 'item_id'], inplace=True)

    resultado = df[df['developer'] == desarrolladora]['sentiment_analysis_2'].value_counts()

    resultado_dic = {
        desarrolladora: [f'Negative = {resultado[0]}', f'Positive = {resultado[2]}']
    }

    return resultado_dic

#-----------------------------------------ENDPOINT 6---------------------------------------#
@app.get('/recomendacion_juego/{id_producto}')
def recomendacion_juego(id_producto: int, n_recom: int = 5, umbral: float = 0.999):
    return item_item_recom(id_producto, n_recom, umbral)

In [3]:
developer('valve')

{'Año 2017': {'Cantidad de Items': 2, 'Contenido Free:': '0.0%'},
 'Año 2016': {'Cantidad de Items': 1, 'Contenido Free:': '100.0%'},
 'Año 2014': {'Cantidad de Items': 7, 'Contenido Free:': '0.0%'},
 'Año 2012': {'Cantidad de Items': 2, 'Contenido Free:': '0.0%'},
 'Año 2011': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2010': {'Cantidad de Items': 2, 'Contenido Free:': '50.0%'},
 'Año 2009': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2008': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2007': {'Cantidad de Items': 3, 'Contenido Free:': '33.33%'},
 'Año 2006': {'Cantidad de Items': 2, 'Contenido Free:': '0.0%'},
 'Año 2005': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2004': {'Cantidad de Items': 5, 'Contenido Free:': '0.0%'},
 'Año 2003': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2001': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2000': {'Cantidad de Items': 2, 'Contenido Free:': '0.0%'},
 'Año

In [10]:
userdata('jonasdbomb')

             user_id  items_count   price  recommend
0            --000--           58  189.96        1.0
1            --ace--           44   19.99        2.0
2          --ionex--           23   19.99        2.0
3       -2SV-vuLB-Kg           68   61.99        5.0
4  -404PageNotFound-          149   59.99        NaN


{'Usuario': 'jonasdbomb',
 'Dinero gastado': '74.76USD',
 '% de recomendación': '0.0 %',
 'Cantidad de Items': 461}

In [16]:
UserForGenre('action')

{'Usuario con mas horas jugadas para el género action:': 'DownSyndromeKid',
 'Horas jugadas:': [{'Año:': 1988, 'Horas:': 11},
  {'Año:': 1990, 'Horas:': 0},
  {'Año:': 1992, 'Horas:': 0},
  {'Año:': 1993, 'Horas:': 0},
  {'Año:': 1994, 'Horas:': 0},
  {'Año:': 1995, 'Horas:': 8},
  {'Año:': 1998, 'Horas:': 0},
  {'Año:': 2000, 'Horas:': 1282},
  {'Año:': 2001, 'Horas:': 1},
  {'Año:': 2002, 'Horas:': 1230},
  {'Año:': 2003, 'Horas:': 1235},
  {'Año:': 2004, 'Horas:': 0},
  {'Año:': 2005, 'Horas:': 0},
  {'Año:': 2006, 'Horas:': 0},
  {'Año:': 2007, 'Horas:': 1231},
  {'Año:': 2008, 'Horas:': 1},
  {'Año:': 2009, 'Horas:': 0},
  {'Año:': 2010, 'Horas:': 1239},
  {'Año:': 2011, 'Horas:': 3716},
  {'Año:': 2012, 'Horas:': 4},
  {'Año:': 2013, 'Horas:': 61},
  {'Año:': 2014, 'Horas:': 234},
  {'Año:': 2015, 'Horas:': 406},
  {'Año:': 2016, 'Horas:': 55},
  {'Año:': 2017, 'Horas:': 96},
  {'Año:': 2018, 'Horas:': 0}]}

In [14]:
df_steam.head()

,id,app_name,title,price,developer,release_date,tag_1980s,tag_1990's,tag_2.5D,tag_2D,...,spec_Single-player,spec_Stats,spec_Steam Achievements,spec_Steam Cloud,spec_Steam Leaderboards,spec_Steam Trading Cards,spec_Steam Turn Notifications,spec_Steam Workshop,spec_SteamVR Collectibles,spec_Valve Anti-Cheat enabled
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,4.99,Kotoshiro,2018-01-04,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,643980,Ironbound,Ironbound,0.00,Secret Level SRL,2018-01-04,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,670290,Real Pool 3D - Poolians,Real Pool 3D - Poolians,0.00,Poolians.com,2017-07-24,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,767400,弹炸人2222,弹炸人2222,0.99,彼岸领域,2017-12-07,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,772540,Battle Royale Trainer,Battle Royale Trainer,3.99,Trickjump Games Ltd,2018-01-04,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
best_developer_year(2015)

{'Puesto 1': 'Valve',
 'Puesto 2': 'Facepunch Studios',
 'Puesto 3': 'Smartly Dressed Games'}

In [25]:
developer_reviews_analysis('Valve')

{'valve': ['Negative = 1095', 'Positive = 6274']}

In [26]:
best_developer_year(2015)

{'Puesto 1': 'Valve',
 'Puesto 2': 'Facepunch Studios',
 'Puesto 3': 'Smartly Dressed Games'}

In [27]:
UserForGenre('adventure')

{'Usuario con mas horas jugadas para el género adventure:': 'Evilutional',
 'Horas jugadas:': [{'Año:': 2009, 'Horas:': 0},
  {'Año:': 2010, 'Horas:': 260},
  {'Año:': 2012, 'Horas:': 10588},
  {'Año:': 2013, 'Horas:': 4},
  {'Año:': 2014, 'Horas:': 34},
  {'Año:': 2017, 'Horas:': 6}]}

In [28]:
userdata('jonasdbomb')

{'Usuario': 'jonasdbomb',
 'Dinero gastado': '74.76USD',
 '% de recomendación': '0.0 %',
 'Cantidad de Items': 461}

In [29]:
developer('kotoshiro')

{'Año 2018': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'}}